# Install the necessaries Libraries

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"


  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-vvhm4281/unsloth_f7820d4a046646d9abe680da4e411d83
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-vvhm4281/unsloth_f7820d4a046646d9abe680da4e411d83
  Resolved https://github.com/unslothai/unsloth.git to commit 6c534341bb229b136f9504443f0161645d2070c5
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [3]:
from torch import __version__; from packaging.version import Version as V
xformers = "xformers==0.0.27" if V(__version__) < V("2.4.0") else "xformers"

In [4]:
!pip install --no-deps {xformers} trl peft accelerate bitsandbytes triton

In [5]:
!pip install unsloth datasets transformers torch

In [6]:
!nvidia-smi

Thu Sep 12 07:15:26 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   56C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

# Import the Necessary Libraries

In [7]:
from unsloth import FastLanguageModel
import torch
import pandas as pd
from datasets import load_dataset, Dataset
from transformers import AutoTokenizer
from transformers import TrainingArguments
from trl import SFTTrainer
from transformers import TextStreamer
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


# Load the Llama-3.1-8B Model and Tokenizer

In [8]:
# Configuration
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.


# Load model and tokenizer
model_name = "unsloth/Meta-Llama-3.1-8B-bnb-4bit"
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit
)


==((====))==  Unsloth 2024.8: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.27.post2. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


# Add LoRA adapters

In [9]:
# Add LoRA adapters
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                    "gate_proj", "up_proj", "down_proj"],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=3407
)

Unsloth 2024.8 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


# Prepare the Dataset

In [10]:
# Function to format the prompts
def formatting_prompts_func(examples):
    texts = []
    for conversation_text, sentiment in zip(examples["conversation_text"], examples["sentiment"]):
        prompt = f"Instruction: Classify the sentiment.\nInput: {conversation_text}\nResponse: {sentiment}"
        text = prompt + tokenizer.eos_token
        texts.append(text)
    return {"text": texts}

df = pd.read_excel('/content/drive/MyDrive/model-save/Dataset-Sentiment Analysis (Task-ML Engineer).xlsx')

# Convert DataFrame to Dataset
data = Dataset.from_pandas(df)

# Apply the formatting function
data = data.map(formatting_prompts_func, batched=True)

Map:   0%|          | 0/99 [00:00<?, ? examples/s]

In [11]:
data

Dataset({
    features: ['conversation_text', 'sentiment', 'text'],
    num_rows: 99
})

In [12]:

# Split dataset into train and test
data = data.train_test_split(test_size=0.2)

# Configure Training Arguments

In [13]:
trainer_args = TrainingArguments(
    per_device_train_batch_size=2,
    # per_device_eval_batch_size=4,
    # gradient_accumulation_steps=4,
    eval_accumulation_steps=2,
    warmup_steps=5,
    max_steps=60,
    learning_rate=2e-4,
    fp16=True,
    logging_steps=1,
    optim="adamw_8bit",
    weight_decay=0.01,
    lr_scheduler_type="linear",
    seed=3407,
    output_dir="outputs"
)


# Train the Model

In [15]:
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=data["train"],
    eval_dataset=data["test"],
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    packing=False,
    args=trainer_args
)

# Train the model
trainer.train()


Map (num_proc=2):   0%|          | 0/79 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/20 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 79 | Num Epochs = 2
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 1
\        /    Total batch size = 2 | Total steps = 60
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,1.391400
2,1.112100
3,1.619700
4,1.277500
5,1.185300
6,1.852000
7,1.149500
8,1.978700
9,1.543800
10,0.954900


TrainOutput(global_step=60, training_loss=0.9483076219757398, metrics={'train_runtime': 163.285, 'train_samples_per_second': 0.735, 'train_steps_per_second': 0.367, 'total_flos': 2132246722142208.0, 'train_loss': 0.9483076219757398, 'epoch': 1.5})

In [16]:
results = trainer.evaluate()
print("Evaluation Results:")
print(results)

Evaluation Results:
{'eval_loss': 0.817980170249939, 'eval_runtime': 10.1862, 'eval_samples_per_second': 1.963, 'eval_steps_per_second': 0.295, 'epoch': 1.5}


# Save the Fine-Tuned Model

In [17]:
model.save_pretrained("lora_model")
tokenizer.save_pretrained("lora_model")


('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/tokenizer.json')

# Let's run the model!

In [18]:
conversation_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

In [19]:
FastLanguageModel.for_inference(model)
inputs = tokenizer(
    [
        conversation_prompt.format(
            "Classify the sentiment of the text.",
            "company_x এজেন্ট থেকে রং নাম্বারে টাকা গেলে ফেরত নেওয়ার কোন উপায় আছে???? বি:দ্র: জিডি করা হয়ছে।যার কাছে টাকা গেছে তার ফোন বন্দ্ব.",
            ""
        )
    ], return_tensors="pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens=64, use_cache=True)
decoded_output = tokenizer.batch_decode(outputs)
decoded_output


['<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nClassify the sentiment of the text.\n\n### Input:\ncompany_x এজেন্ট থেকে রং নাম্বারে টাকা গেলে ফেরত নেওয়ার কোন উপায় আছে???? বি:দ্র: জিডি করা হয়ছে।যার কাছে টাকা গেছে তার ফোন বন্দ্ব.\n\n### Response:\nnegative<|end_of_text|>']